In [2]:
import os
import sys
import numpy as np
sys.path.append(os.path.join("../"))
from orquestra.qml.models.tn.mps import (
    MPS,
    AMPS,
    EqualityConstrainedMps,
    OneSiteDMRG,
    RandCircuitInitializer,
    TwoSiteDMRG,
)



from orquestra.qml.constraints import AxbEqualityConstraint
from orquestra.qml.data_loaders import BarsAndStripesDataLoader, new_data_loader
from orquestra.qml.trainers import SimpleTrainer
import selfies as sf
from rdkit import Chem
# from utils.chem_utils import (SelfiesEncoding)
from orquestra.qml.data_loaders import new_data_loader
import torch
from orquestra.qml.models.samplers.jx import RandomCardinalitySampler 
from orquestra.qml.data_loaders import new_data_loader


# Simple Example 



In [2]:
n_x = 8
n_y = 8
n_sites = n_x * n_y

mps_gen = MPS(
    [2 for _ in range(n_sites)],
    initializer=RandCircuitInitializer(
        bond_dim=32,
        element_type="Float32",
    ),
    scheme=OneSiteDMRG(),
    output_type="Int32",
)

In [ ]:
np.hstack((np.ones(8), np.zeros(8)))

array([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [2]:
n_x=8
n_y=8
# constraint = AxbEqualityConstraint(np.matrix(np.hstack((np.ones(8), np.zeros(8))), np.hstack((np.zeros(8), np.ones(8)))),np.ones(1, 1))
constraint1 = AxbEqualityConstraint(np.hstack((np.ones(8), np.zeros(8))),np.ones(1))
constraint2 = AxbEqualityConstraint(np.hstack((np.zeros(8), np.ones(8))),np.ones(1))
# filter out those that don't meet equality constraint

init_sample_first= RandomCardinalitySampler(16//2,1).generate(4)
init_sample_second= RandomCardinalitySampler(16//2,1).generate(4)


initial_samples_list = []




for first, second in zip(init_sample_first,init_sample_second):
    initial_samples_list.append(list(first)+list(second))

init_sample_array = np.asarray(initial_samples_list)



In [3]:
init_sample_array.shape[0]

4

In [3]:
dl=new_data_loader(torch.tensor(init_sample_array),batch_size=-1).shuffle(123456)

In [4]:



# dl = BarsAndStripesDataLoader(n_x, n_y, batch_size=-1).filter(constraint1)

n_sites = 16
mps_gen = MPS(
    [2 for _ in range(n_sites)],
    initializer=RandCircuitInitializer(
        bond_dim=32,
        element_type="Float32",
    ),
    scheme=OneSiteDMRG(),
    output_type="Int32",
)

# assert len(mps_gen.tensors) == n_sites, "MPS must have n_sites tensors"
# assert (
#     len(mps_gen.bond_dimensions) == n_sites - 1
# ), "MPS must have n_sites-1 bond dimensions"

SimpleTrainer().train(
    mps_gen,
    dl,
    2,
)

samples = mps_gen.generate(10, 10)

Epoch 1 / 2:   0%|          | 0/1 [00:00<?, ?steps/s]

TypeError: Julia: MethodError: no method matching train_on_batch(::ITensors.MPS, ::PythonCall.PyArray{Int64, 2, true, false, Int64}, ::PythonCall.PyArray{PythonCall.Py, 0, true, true, PythonCall.UnsafePyObject}, ::Main.QML.TN.OneSiteDMRG)
Closest candidates are:
  train_on_batch(::ITensors.MPS, ::PythonCall.PyArray{<:Integer}, !Matched::PythonCall.PyArray{<:AbstractFloat}, ::T) where T<:Main.QML.TN.Scheme at ~/workspace/repos/orquestra-qml-core/src/python/orquestra/qml/jl/QML.jl/src/TN/mps.jl:101
  train_on_batch(!Matched::Main.QML.TN.AMPS, ::PythonCall.PyArray{<:Integer}, !Matched::PythonCall.PyArray{<:AbstractFloat}) at ~/workspace/repos/orquestra-qml-core/src/python/orquestra/qml/jl/QML.jl/src/TN/amps.jl:86

In [7]:
np.sum(samples,axis=1)

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

# load Dataset

In [2]:
path_to_dataset = "../data/KRAS_G12D/KRAS_G12D_inhibitors_update2023.csv"


In [3]:
selfies = SelfiesEncoding(path_to_dataset, dataset_identifier="insilico_KRAS")


In [4]:
data = selfies.encoded_samples

In [5]:
dataloader = new_data_loader(
    data=data,probs = torch.ones(len(data)) / len(data), batch_size=-1
).shuffle(1234)

In [11]:
selfies.char_to_index

{'[Branch2_3]': 0,
 '[Branch1_2]': 1,
 '[Branch1_3]': 2,
 '[Branch2_2]': 3,
 '[Expl=Ring1]': 4,
 '[S]': 5,
 '[NHexpl]': 6,
 '[F]': 7,
 '[Expl=Ring2]': 8,
 '[Cl]': 9,
 '[N]': 10,
 '[^]': 11,
 '[C@@expl]': 12,
 '[Siexpl]': 13,
 '[Br]': 14,
 '[O]': 15,
 '[C@expl]': 16,
 '[Ring2]': 17,
 '[#N]': 18,
 '[C@@Hexpl]': 19,
 '[Branch2_1]': 20,
 '[C@Hexpl]': 21,
 '[C]': 22,
 '[Branch1_1]': 23,
 '[P]': 24,
 '[=C]': 25,
 '[=N]': 26,
 '[Ring1]': 27,
 '[Hexpl]': 28,
 '[=O]': 29,
 '[nop]': 30,
 '[#C]': 31,
 '[N@H+expl]': 32,
 '[2Hexpl]': 33}

# MPS

In [16]:
dim_size = 6
n_dim = 2
amps_gen = AMPS(
        [dim_size for _ in range(n_dim)],
        initializer=RandCircuitInitializer(
            bond_dim=32,
            element_type="Float32",
        )
    )

In [17]:
amps_gen.generate(2)

array([[5, 2],
       [1, 2]], dtype=int32)

In [ ]:
dataloader = new_data_loader(
    data=data,probs = torch.ones(len(data)) / len(data), batch_size=-1
).shuffle(1234)

In [20]:
SimpleTrainer().train(
        amps_gen,
        dataloader,
        2,
    )

Epoch 1 / 2: 100%|██████████| 1/1 [47:40<00:00, 2860.31s/steps, grad_norm=3.8538E-01, training_loss=2.9446E+03, time_get_data=3.6124E-01, time_optimization=2.8597E+03]


KeyboardInterrupt: 

In [11]:

selfies.decode_fn(amps_gen.generate(1000))

['[C@@H]=[N@H+]N=O',
 'Br[Si][H]',
 'Cl[Si]Br',
 '[C@]1=C[Si](OON1[H])O[2H]',
 'BrNF',
 'CBr',
 'C(C)([2H])Cl',
 '[N@H+]=NSF',
 'C#N',
 'BrBr',
 'S#SCl',
 '[N@H+]P[Si]#N',
 '[C@][NH]CF',
 '[C@]([C@@H]([C@@]NC([H])))=P[NH][H]',
 'N[C@@](O)[Si]=N[NH]N=NNC(Br)Cl',
 'NCl',
 '[C@](N)#COC(OF)[N@H+]Cl',
 '[H]Br',
 'Cl[Si](S#[C@]SF)([H])#N',
 'C[H]',
 '[2H][2H]',
 'C([C@@H]=N[H])[NH]P#CBr',
 'N(N[H])NN[C@@]([NH])(NON=CNP(O)C=N)[N@H+]S#C[2H]',
 'C([C@@H])N[2H]',
 '[N@H+]=C(OPOO[C@@H]=C([C@][NH]Cl)[Si]OBr)F',
 'CC[H]',
 '[C@]1#[C@@][C@H][C@]([N@H+][C@@H][Si]1[2H])[2H]',
 '[N@H+][Si][C@@]Cl',
 'F[2H]',
 'N[Si][C@H][2H]',
 '[C@H]#N',
 '[Si][C@@H][N@H+](F)[NH]C[N@H+](S([H]))P[Si][C@@][C@][C@@][N@H+]S([Si])[2H]',
 'P[C@H]C([C@H])F',
 '[C@]#C[2H]',
 'N(NOCl)O[C@@][2H]',
 'O[Si][C@@]OCBr',
 '[Si][C@@]#[Si]N=N[NH]S=O',
 'C1([H])[C@]N(N(C[C@](F))Cl)P([C@]1=C[NH]SS[C@@H]F)(O)([NH])[N@H+][2H]',
 'P#C[C@H][N@H+]Br',
 '[NH][H]',
 'SBr',
 'S=C[H]',
 '[C@@][N@H+][C@@H]P#N',
 '[C@]=O',
 '[C@@][C@@H]Br',
 'N(=C

# MPS 


In [19]:

n_sites = len(data[0])
max_int = max(data[0])
mps_gen = MPS(
        [max_int for _ in range(n_sites)],
        initializer=RandCircuitInitializer(
            bond_dim=32,
            element_type="Float32",
        ),
        scheme=OneSiteDMRG(),
    )
# n_x = 4
# n_y = 4
# n_sites = n_x * n_y

# mps_gen = MPS(
#     [4 for _ in range(n_sites)],
#     initializer=RandCircuitInitializer(
#         bond_dim=32,
#         element_type="Float32",
#     ),
#     scheme=OneSiteDMRG(),
# )


In [20]:
SimpleTrainer().train(
        mps_gen,
        dataloader,
        2,
    )
# SimpleTrainer().train(
#     mps_gen,
#     BarsAndStripesDataLoader(n_x, n_y, batch_size=-1),
#     2,
# )

JuliaError: TaskFailedException
Stacktrace:
  [1] wait
    @ ./task.jl:345 [inlined]
  [2] threading_run(fun::Main.QML.TN.var"#23#threadsfor_fun#6"{Main.QML.TN.var"#23#threadsfor_fun#4#7"{PythonCall.PyArray{Int64, 2, true, false, Int64}, PythonCall.PyArray{Float32, 1, true, true, Float32}, Vector{Main.QML.TN.MPSProbPair}, UnitRange{Int64}}}, static::Bool)
    @ Base.Threads ./threadingconstructs.jl:38
  [3] macro expansion
    @ ./threadingconstructs.jl:89 [inlined]
  [4] data_to_tensors
    @ ~/workspace/libs/orquestra-qml-core/src/python/orquestra/qml/jl/QML.jl/src/TN/data_preparation.jl:57 [inlined]
  [5] optimize(scheme::Main.QML.TN.OneSiteDMRG, mps::ITensors.MPS, data_samples::PythonCall.PyArray{Int64, 2, true, false, Int64}, probs::PythonCall.PyArray{Float32, 1, true, true, Float32})
    @ Main.QML.TN ~/workspace/libs/orquestra-qml-core/src/python/orquestra/qml/jl/QML.jl/src/TN/mps.jl:155
  [6] macro expansion
    @ ~/workspace/libs/orquestra-qml-core/src/python/orquestra/qml/jl/

In [ ]:
samples = mps_gen.generate(1, 10)

In [ ]:
selfies.decode_fn(samples)

array([[False, False, False, False, False, False, False, False,  True,
         True, False, False,  True, False, False,  True]])